In [26]:
from pulp import *

proces_time = {'A1':{'A':6,'B':2,'C':6,'D': 4,'E':5},
               'A2':{'A':10,'B':8,'C':4,'D':2,'E':3},
               'A3':{'A':4,'B':3,'C':5,'D':7,'E':4}}

# 
N = len(proces_time.keys()) 
M = len(proces_time['A1'].keys())

# convert data to dictionary {(i,k): value}   (i=job, k=machine)
p = {(i+1,j+1): proces_time['A'+str(i+1)][chr(j+65)] for i in range(N) for j in range(M)}

jobs = range(1,N+1)
machines = range(1,M+1)

prob = LpProblem("jobshop", LpMinimize)

# Variables
x = LpVariable.dicts("pos", ((i,k) for i in range(N+1) for k in range(N+1)), cat='Binary')
s = LpVariable.dicts("time", ((k,j) for k in range(N+1) for j in range(M+1)), lowBound=0)

# One job is scheduled only once
for i in jobs: 
    prob += lpSum(x[(i,k)] for k in jobs) == 1

# Each possible order is used only once
for k in jobs: 
    prob += lpSum(x[(i,k)] for i in jobs) == 1

# Initial conditions
for j in machines:
    prob += s[0,j] == 0  # Dummy job 0 finishes at time 0

for k in jobs:
    prob += s[k,0] == 0  # Dummy machine 0 has zero processing time

# First machine (A1) timing constraints
for k in range(1, N+1):
    prob += s[k,1] == s[k-1,1] + lpSum(x[(i,k)] * p[(i,1)] for i in jobs)

# Second machine (A2) timing constraints (take into acount the maximum betwen the previous machine of the same job and the next machine of the previous job)
for j in range(2, M+1):
    prob += s[k,j] >= s[k,j-1] + lpSum(x[(i,k)] * p[(i,j)] for i in jobs)
    prob += s[k,j] >= s[k-1,j] + lpSum(x[(i,k)] * p[(i,j)] for i in jobs)

# Objective: Minimize makespan
prob += s[N,M]

# Solve the problem
prob.solve(PULP_CBC_CMD(msg=False))

# Print objective function value
print("Objective value: ", value(prob.objective))

'''
# Print the results
for v in prob.variables():
    print(v.name, "=", v.varValue)
'''
# Decode to original machine names and job names
solution = {i: chr(j+65) for (i,j), v in x.items() if v.varValue == 1 and i > 0 and j > 0}
print(solution)



Objective value:  37.0
{1: 'B', 2: 'D', 3: 'C'}
